# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 20 2022 8:44AM,242292,10,EXP8235845,"Citieffe, Inc.",Released
1,Jul 20 2022 8:44AM,242291,19,ELI72122,Eli Lilly and Company,Released
2,Jul 20 2022 8:43AM,242289,10,0085890208,ISDIN Corporation,Released
3,Jul 20 2022 8:43AM,242289,10,0085890193,ISDIN Corporation,Released
4,Jul 20 2022 8:43AM,242289,10,0085891185,ISDIN Corporation,Released
5,Jul 20 2022 8:43AM,242289,10,0085889806,ISDIN Corporation,Released
6,Jul 20 2022 8:43AM,242289,10,0085889807,ISDIN Corporation,Released
7,Jul 20 2022 8:43AM,242289,10,0085889826,ISDIN Corporation,Released
8,Jul 20 2022 8:43AM,242289,10,0085889833,ISDIN Corporation,Released
9,Jul 20 2022 8:43AM,242289,10,0085889829,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,242287,Released,31
34,242289,Released,25
35,242290,Released,1
36,242291,Released,1
37,242292,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242287,NaN,NaN,31.0
242289,NaN,NaN,25.0
242290,NaN,NaN,1.0
242291,NaN,NaN,1.0
242292,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242201,0.0,0.0,14.0
242207,0.0,0.0,1.0
242209,0.0,1.0,0.0
242224,0.0,0.0,1.0
242229,6.0,6.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242201,0,0,14
242207,0,0,1
242209,0,1,0
242224,0,0,1
242229,6,6,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242201,0,0,14
1,242207,0,0,1
2,242209,0,1,0
3,242224,0,0,1
4,242229,6,6,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242201,,,14
1,242207,,,1
2,242209,,1,
3,242224,,,1
4,242229,6,6,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 20 2022 8:44AM,242292,10,"Citieffe, Inc."
1,Jul 20 2022 8:44AM,242291,19,Eli Lilly and Company
2,Jul 20 2022 8:43AM,242289,10,ISDIN Corporation
27,Jul 20 2022 8:43AM,242290,16,Seqirus
28,Jul 20 2022 8:37AM,242287,10,ISDIN Corporation
59,Jul 20 2022 8:37AM,242285,10,ISDIN Corporation
103,Jul 20 2022 8:35AM,242283,10,ISDIN Corporation
160,Jul 20 2022 8:32AM,242286,16,Sartorius Bioprocess Solutions
161,Jul 20 2022 8:31AM,242282,16,Seqirus
162,Jul 20 2022 8:30AM,242267,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 20 2022 8:44AM,242292,10,"Citieffe, Inc.",,,1
1,Jul 20 2022 8:44AM,242291,19,Eli Lilly and Company,,,1
2,Jul 20 2022 8:43AM,242289,10,ISDIN Corporation,,,25
3,Jul 20 2022 8:43AM,242290,16,Seqirus,,,1
4,Jul 20 2022 8:37AM,242287,10,ISDIN Corporation,,,31
5,Jul 20 2022 8:37AM,242285,10,ISDIN Corporation,,,44
6,Jul 20 2022 8:35AM,242283,10,ISDIN Corporation,,,57
7,Jul 20 2022 8:32AM,242286,16,Sartorius Bioprocess Solutions,,,1
8,Jul 20 2022 8:31AM,242282,16,Seqirus,,,1
9,Jul 20 2022 8:30AM,242267,10,ISDIN Corporation,,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 8:44AM,242292,10,"Citieffe, Inc.",1,,
1,Jul 20 2022 8:44AM,242291,19,Eli Lilly and Company,1,,
2,Jul 20 2022 8:43AM,242289,10,ISDIN Corporation,25,,
3,Jul 20 2022 8:43AM,242290,16,Seqirus,1,,
4,Jul 20 2022 8:37AM,242287,10,ISDIN Corporation,31,,
5,Jul 20 2022 8:37AM,242285,10,ISDIN Corporation,44,,
6,Jul 20 2022 8:35AM,242283,10,ISDIN Corporation,57,,
7,Jul 20 2022 8:32AM,242286,16,Sartorius Bioprocess Solutions,1,,
8,Jul 20 2022 8:31AM,242282,16,Seqirus,1,,
9,Jul 20 2022 8:30AM,242267,10,ISDIN Corporation,13,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 8:44AM,242292,10,"Citieffe, Inc.",1,,
1,Jul 20 2022 8:44AM,242291,19,Eli Lilly and Company,1,,
2,Jul 20 2022 8:43AM,242289,10,ISDIN Corporation,25,,
3,Jul 20 2022 8:43AM,242290,16,Seqirus,1,,
4,Jul 20 2022 8:37AM,242287,10,ISDIN Corporation,31,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 8:44AM,242292,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Jul 20 2022 8:44AM,242291,19,Eli Lilly and Company,1.0,NaN,NaN
2,Jul 20 2022 8:43AM,242289,10,ISDIN Corporation,25.0,NaN,NaN
3,Jul 20 2022 8:43AM,242290,16,Seqirus,1.0,NaN,NaN
4,Jul 20 2022 8:37AM,242287,10,ISDIN Corporation,31.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 8:44AM,242292,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Jul 20 2022 8:44AM,242291,19,Eli Lilly and Company,1.0,0.0,0.0
2,Jul 20 2022 8:43AM,242289,10,ISDIN Corporation,25.0,0.0,0.0
3,Jul 20 2022 8:43AM,242290,16,Seqirus,1.0,0.0,0.0
4,Jul 20 2022 8:37AM,242287,10,ISDIN Corporation,31.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4360892,271.0,0.0,0.0
16,726858,3.0,0.0,0.0
18,484493,2.0,0.0,0.0
19,1695783,22.0,9.0,6.0
20,726643,33.0,1.0,0.0
22,484549,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4360892,271.0,0.0,0.0
1,16,726858,3.0,0.0,0.0
2,18,484493,2.0,0.0,0.0
3,19,1695783,22.0,9.0,6.0
4,20,726643,33.0,1.0,0.0
5,22,484549,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,271.0,0.0,0.0
1,16,3.0,0.0,0.0
2,18,2.0,0.0,0.0
3,19,22.0,9.0,6.0
4,20,33.0,1.0,0.0
5,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,271.0
1,16,Released,3.0
2,18,Released,2.0
3,19,Released,22.0
4,20,Released,33.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20,22
Status,,,,,,
Completed,0.0,0.0,0.0,6.0,0.0,0.0
Executing,0.0,0.0,0.0,9.0,1.0,1.0
Released,271.0,3.0,2.0,22.0,33.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20,22
0,Completed,0.0,0.0,0.0,6.0,0.0,0.0
1,Executing,0.0,0.0,0.0,9.0,1.0,1.0
2,Released,271.0,3.0,2.0,22.0,33.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20,22
0,Completed,0.0,0.0,0.0,6.0,0.0,0.0
1,Executing,0.0,0.0,0.0,9.0,1.0,1.0
2,Released,271.0,3.0,2.0,22.0,33.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()